In [1]:
import os

In [2]:
%pwd

'/Users/aksshar/PycharmProjects/End-to-end-Machine-Learning-Project-with-MLflow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/aksshar/PycharmProjects/End-to-end-Machine-Learning-Project-with-MLflow'

In [5]:
import pandas as pd

In [6]:
data = pd.read_csv("artifacts/data_ingestion/tango_data.csv")
data.head()

,CUSCODE,ITEMCODE,SALEQTY,SHOPCODE,SALEAMNT,MASTERITEMCODE,SUPPLIERCODE,CAT1CODE,CAT2CODE,CAT3CODE,BUYERCODE,PROFCODE,GROUPPROFCODE,profile
0,465339,244860,6,47,11.39,1854,285,20,10,9,15,3,2,1
1,694510,244860,8,18,15.19,1854,285,20,10,9,15,3,2,1
2,296440,244860,12,36,22.78,1854,285,20,10,9,15,3,2,1
3,902877,244860,1,12,1.90,1854,285,20,10,9,15,2,3,1
4,306992,244860,2,24,3.80,1854,285,20,10,9,15,4,2,1


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175010 entries, 0 to 175009
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   CUSCODE         175010 non-null  int64  
 1   ITEMCODE        175010 non-null  int64  
 2   SALEQTY         175010 non-null  int64  
 3   SHOPCODE        175010 non-null  int64  
 4   SALEAMNT        175010 non-null  float64
 5   MASTERITEMCODE  175010 non-null  int64  
 6   SUPPLIERCODE    175010 non-null  int64  
 7   CAT1CODE        175010 non-null  int64  
 8   CAT2CODE        175010 non-null  int64  
 9   CAT3CODE        175010 non-null  int64  
 10  BUYERCODE       175010 non-null  int64  
 11  PROFCODE        175010 non-null  int64  
 12  GROUPPROFCODE   175010 non-null  int64  
 13  profile         175010 non-null  int64  
dtypes: float64(1), int64(13)
memory usage: 18.7 MB


In [8]:
data.isnull().sum()

CUSCODE           0
ITEMCODE          0
SALEQTY           0
SHOPCODE          0
SALEAMNT          0
MASTERITEMCODE    0
SUPPLIERCODE      0
CAT1CODE          0
CAT2CODE          0
CAT3CODE          0
BUYERCODE         0
PROFCODE          0
GROUPPROFCODE     0
profile           0
dtype: int64

In [9]:
data.shape

(175010, 14)

In [10]:
data.columns

Index(['CUSCODE', 'ITEMCODE', 'SALEQTY', 'SHOPCODE', 'SALEAMNT',
       'MASTERITEMCODE', 'SUPPLIERCODE', 'CAT1CODE', 'CAT2CODE', 'CAT3CODE',
       'BUYERCODE', 'PROFCODE', 'GROUPPROFCODE', 'profile'],
      dtype='object')

In [11]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [12]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [14]:
import os
from mlProject import logger

In [15]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e



In [17]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2023-09-19 18:57:16,793: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-09-19 18:57:16,796: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-19 18:57:16,799: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-19 18:57:16,800: INFO: common: created directory at: artifacts]
[2023-09-19 18:57:16,801: INFO: common: created directory at: artifacts/data_validation]
